In [1]:
import argparse
from collections import Counter
import code
import os
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [3]:
import numpy as np
import pandas as pd
import pkuseg
from elmoformanylangs import Embedder

In [4]:
e=Embedder('./zhs.model')
seg=pkuseg.pkuseg()


2020-01-02 21:06:07,999 INFO: char embedding size: 6169
2020-01-02 21:06:08,449 INFO: word embedding size: 71222
2020-01-02 21:06:11,989 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

In [18]:
# sents = ["今天天气真好啊", "潮水退了就知道谁没穿裤子"]
# sents = [seg.cut(sent) for sent in sents]
# print(sents)

[['今天', '天气', '真', '好', '啊'], ['潮水', '退', '了', '就', '知道', '谁', '没', '穿', '裤子']]


In [12]:
# embeddings = e.sents2elmo(sents)

2019-12-31 13:29:57,075 INFO: 1 batches, avg len: 9.0


In [16]:
# print(len(embeddings[0][0]))

1024


In [7]:
def main():
    # for bash command
    #parser=argparse.ArgumentParser()
    #parser.add_argument("--train_file",default=None,type=str,required=True,
    #                   help="my train file")
    #parser.add_argument("--batch_size",default=64,type=int,required=False,
    #                   help="my batch size")
    #args=parser.parse_args()
    
    if not os.path.exists("embeddings.pkl"):
        # train_df = pd.read_csv(args.train_file)  for bash command
        train_df=pd.read_csv("/Users/valleria_ruka/Desktop/FAQ/lawzhidao_filter.csv")
        candidates=train_df[train_df["is_best"]==1][["title","reply"]]
        candidate_title=candidates["title"].tolist()
        candidate_reply=candidates["reply"].tolist()
        
        titles=[seg.cut(title) for title in candidates["title"]]
        embeddings=e.sents2elmo(titles)
        
        #可以不求平均,直接embedding送入
        candidate_embeddings=[np.mean(embedding,0) for embedding in embeddings]
        
        with open("embeddings.pkl","wb") as fout:
            # pickle.dump([candidate_title,candidate_reply,embeddings],fout)
            pickle.dump([candidate_title,candidate_reply,candidate_embeddings],fout)
    
    else:
         with open("embeddings.pkl","rb") as fin:
                #不求平均
                # candidate_title,candidate_reply,embeddings=pickle.load(fin)
                candidate_title,candidate_reply,candidate_embeddings=pickle.load(fin)
            
    
    while True:
        title=input("你丫的问题是? \n")
        if len(title.strip())==0:
            continue
        
        title=[seg.cut(title.strip())]
        title_embedding=[np.mean(e.sents2elmo(title)[0],0)]
        
        #不求平均
        # scores=cosine_similarity(title_embedding,embeddings)[0]
        scores=cosine_similarity(title_embedding,candidate_embeddings)[0]
        top5_indices=scores.argsort()[-5:][::-1]
        for index in top5_indices:
            print(print("可能的答案，参考问题：" + candidate_title[index] + 
                        "\t答案：" + candidate_reply[index] + "\t得分：" + str(scores[index])))
        

In [8]:
main()

2020-01-02 21:13:58,665 INFO: 286 batches, avg len: 12.7
2020-01-02 21:14:37,711 INFO: Finished 1000 sentences.
2020-01-02 21:15:19,734 INFO: Finished 2000 sentences.
2020-01-02 21:15:57,194 INFO: Finished 3000 sentences.
2020-01-02 21:16:34,727 INFO: Finished 4000 sentences.
2020-01-02 21:17:19,037 INFO: Finished 5000 sentences.
2020-01-02 21:17:54,792 INFO: Finished 6000 sentences.
2020-01-02 21:18:39,572 INFO: Finished 7000 sentences.
2020-01-02 21:19:19,971 INFO: Finished 8000 sentences.
2020-01-02 21:20:09,771 INFO: Finished 9000 sentences.
2020-01-02 21:21:02,686 INFO: Finished 10000 sentences.
2020-01-02 21:21:46,379 INFO: Finished 11000 sentences.
2020-01-02 21:22:21,164 INFO: Finished 12000 sentences.
2020-01-02 21:23:13,519 INFO: Finished 13000 sentences.
2020-01-02 21:23:54,101 INFO: Finished 14000 sentences.
2020-01-02 21:24:37,112 INFO: Finished 15000 sentences.
2020-01-02 21:25:22,646 INFO: Finished 16000 sentences.
2020-01-02 21:26:11,474 INFO: Finished 17000 sentences.


你丫的问题是? 
酒驾怎么办?


2020-01-02 21:28:53,391 INFO: 1 batches, avg len: 5.0


可能的答案，参考问题：修车协商不成怎么办?	答案：根据《侵权责任法》的规定，发生财产损失双方协商不成的可以到法院起诉，由法院判决。	得分：0.8282025
None
可能的答案，参考问题：360.借钱是真的吗?不是高利贷坑人的?	答案：民间借贷是一种民事法律行为合法的在法律上认定的高利。贷是指超过银行同期基准利率的4倍，也就是产生法律纠纷时支持同期基准利率四倍以内的利率水平，超过银行同期基准利率的4倍，即为高利。贷，不受法律保护。因为不同期限的基准利率不同，高利，贷的法律认定还与期限相关。同期基准利率是指中央银行公布的基准利率，如一年期，基准利率为6%，4倍为24%，即超过24%为法律认定的高利，贷。就一年期来说，1####元，年利息不超过24%*1####=24##元不算高，利贷。	得分：0.8056763
None
可能的答案，参考问题：喝醉酒取保后审还用做牢么?	答案：要结合相关案情。喝醉酒犯了什么罪，要说清楚。喝醉酒不犯罪啊	得分：0.7995957
None
可能的答案，参考问题：小猪白卡算是高利贷吗?	答案：在法律上认定的高利。贷是指超过银行同期基准利率的4倍，也就是产生法律纠纷时支持同期基准利率四倍以内的利率水平，超过银行同期基准利率的4倍，即为高利。贷，不受法律保护。因为不同期限的基准利率不同，高利，贷的法律认定还与期限相关。同期基准利率是指中央银行公布的基准利率，如一年期，基准利率为6%，4倍为24%，即超过24%为法律认定的高利，贷。就一年期来说，1####元，年利息不超过24%*1####=24##元不算高，利贷。	得分：0.7940981
None
可能的答案，参考问题：房管局查房需要户口本吗?	答案：查别人，除非有委托书或者公正书，否则不会让你查的。查自己，就是本人身份证就可以了。	得分：0.7617479
None
你丫的问题是? 
怎样找律师?


2020-01-02 21:29:08,015 INFO: 1 batches, avg len: 6.0


...

2020-01-02 21:31:10,285 INFO: 1 batches, avg len: 6.0


可能的答案，参考问题：给私人老板做工受伤?怎么赔偿,去找那个单位?	答案：打工受伤，可以选择私了，协商解决。如果不能协商，可以选择报警，需要到当地公安局做法医鉴定，再做赔偿	得分：0.8056358
None
可能的答案，参考问题：北京金石律师事务所怎么样？收费贵吗？如何找律师？	答案：各地区有一定差异，大致如下：无财产争议案件：普通民事、经济、行政案件，不涉及财产的，根据案件性质、复杂程度、工作所需耗费时间等因素，在6###—1#####元之间协商收取；外地民事、经济、行政案件不涉及财产的，代理费不低于2####元；律师代理分为以下几种第一种：一般代理。依律师职业道德为当事人代理案件，维护当事人权利。一般为纠纷标的的3-5%。标的高的话也可以低于该比例。第二种：风险代理。按胜诉金额或以得到的金额的百分比支付代理费，或按减少的支付额的百分比支付代理费，比例当然高于一般代理。第三种：半风险代理。给一定的基本费用，其他的再按风险代理计算，当然就介于一般代理和风险代理的比例之间。	得分：0.80096304
None
可能的答案，参考问题：咨询律师怎么找？	答案：咨询可分为：现场口头咨询、当事人提供案件材料后律师提供咨询意见及法律方案；法律咨询是指提供法律知识问题的解答。由于法律的复杂性，非专业人士在遇到法律问题时，往往需要求助于律师一类的法律专业人士。律师收咨询费按计时收费，根据案件的复杂性和工作量及各地区不一样，知名度高的收费会高点，这个你要和律师协商，一般都是协商制。通常会根据问题的难易程度及回答问题所耗费的时间、精力评估收取费用。	得分：0.7905098
None
可能的答案，参考问题：浠水人我想问下起诉离婚要怎么处理需要找律师吗	答案：委托律师是公民的权利，律师费用参考：第一种：一般代理。依律师职业道德为当事人代理案件，维护当事人权利。一般为纠纷标的的3-5%。标的高的话也可以低于该比例。第二种：风险代理。按胜诉金额或以得到的金额的百分比支付代理费，或按减少的支付额的百分比支付代理费，比例当然高于一般代理。第三种：半风险代理。给一定的基本费用，其他的再按风险代理计算，当然就介于一般代理和风险代理的比例之间。	得分：0.78774285
None
可能的答案，参考问题：想找个律师，哪里的律师好，怎么收费的	答案：各地区有一定差异，大致如下：无财产争议案件：普通民

2020-01-02 21:31:36,567 INFO: 1 batches, avg len: 6.0


可能的答案，参考问题：2018无证醉驾新规?无事故,二轮摩托车?	答案：根据最*人*法*等颁布的《关于办理醉酒驾驶机动车刑事案件适用法律若干问题的意见》规定：一、在道路上驾驶机动车，血液酒精含量达到80毫克/100毫升以上的，属于醉酒驾驶机动车，依照刑法第一百三十三条之一第一款的规定，以危险驾驶罪定罪处罚。前款规定的“道路”“机动车”，适用道路交通安全法的有关规定。二、醉酒驾驶机动车，具有下列情形之一的，依照刑法第一百三十三条之一第一款的规定，从重处罚：(一)造成交通事故且负事故全部或者主要责任，或者造成交通事故后逃逸，尚未构成其他犯罪的;(二)血液酒精含量达到200毫克/100毫升以上的;(三)在*速公路、城市快速路上驾驶的;(四)驾驶载有乘客的营运机动车的;(五)有严重超员、超载或者超速驾驶，无驾驶资格驾驶机动车，使用伪造或者变造的机动车牌证等严重违反道路交通安全法的行为的;(六)逃避公安机关依法检查，或者拒绝、阻碍公安机关依法检查尚未构成其他犯罪的;(七)曾因酒后驾驶机动车受过行政处罚或者刑事追究的;(八)其他可以从重处罚的情形。三、醉酒驾驶机动车，以暴力、威胁方法阻碍公安机关依法检查，又构成妨害公务罪等其他犯罪的，依照数罪并罚的规定处罚。四、对醉酒驾驶机动车的被告人判处罚金，应当根据被告人的醉酒程度、是否造成实际损害、认罪悔罪态度等情况，确定与主刑相适应的罚金数额。五、公安机关在查处醉酒驾驶机动车的犯罪嫌疑人时，对查获经过、呼气酒精含量检验和抽取血样过程应当制作记录;有条件的，应当拍照、录音或者录像;有证人的，应当收集证人证言。六、血液酒精含量检验鉴定意见是认定犯罪嫌疑人是否醉酒的依据。犯罪嫌疑人经呼气酒精含量检验达到本意见第一条规定的醉酒标准，在抽取血样之前脱逃的，可以以呼气酒精含量检验结果作为认定其醉酒的依据。犯罪嫌疑人在公安机关依法检查时，为逃避法律追究，在呼气酒精含量检验或者抽取血样前又饮酒，经检验其血液酒精含量达到本意见第一条规定的醉酒标准的，应当认定为醉酒。七、办理醉酒驾驶机动车刑事案件，应当严格执行刑事诉讼法的有关规定，切实保障犯罪嫌疑人、被告人的诉讼权利，在法定诉讼期限内及时侦查、起诉、审判。对醉酒驾驶机动车的犯罪嫌疑人、被告人，根据案件情况，可以拘留或者取保候审。对符合取保候审条件，但犯罪嫌疑人、被告人不能提出保证人，也不交纳保证金的，可以

KeyboardInterrupt: 

array([[0.99258333, 0.61178435]])